In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [70]:
data= pd.read_csv(r'C:\Users\48797\ML\kaggle\insurance\train.csv')

In [71]:
data =data.drop('Id', axis=1)

In [72]:
data['T1_V4'] = pd.factorize(data['T1_V4'])[0]
data['T1_V5'] = pd.factorize(data['T1_V5'])[0]
data['T1_V6'] = pd.factorize(data['T1_V6'])[0]
data['T1_V7'] = pd.factorize(data['T1_V7'])[0]
data['T1_V8'] = pd.factorize(data['T1_V8'])[0]
data['T1_V9'] = pd.factorize(data['T1_V9'])[0]
data['T1_V11'] = pd.factorize(data['T1_V11'])[0]
data['T1_V12'] = pd.factorize(data['T1_V12'])[0]
data['T1_V15'] = pd.factorize(data['T1_V15'])[0]
data['T1_V16'] = pd.factorize(data['T1_V16'])[0]
data['T1_V17'] = pd.factorize(data['T1_V17'])[0]

data['T2_V3'] = pd.factorize(data['T2_V3'])[0]
data['T2_V5'] = pd.factorize(data['T2_V5'])[0]
data['T2_V11'] = pd.factorize(data['T2_V11'])[0]
data['T2_V12'] = pd.factorize(data['T2_V12'])[0]
data['T2_V13'] = pd.factorize(data['T2_V13'])[0]

In [73]:
import xgboost as xgb
X= data.iloc[:,1:]
y=data.iloc[:,0]


In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [75]:

from xgboost import XGBRegressor
xg_reg = XGBRegressor(colsample_bytree=0.7, max_depth=5, reg_alpha=10, n_estimators= 10)
xg_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [76]:
y_pred = xg_reg.predict(X_test)

In [77]:
from sklearn.metrics import mean_squared_error

rmse =mean_squared_error(y_test, y_pred, squared= False)
print('RMSE: ' f'{rmse}')

RMSE: 3.870445404467488


In [78]:
from sklearn.feature_selection import SelectFromModel

selection = SelectFromModel(xg_reg, threshold= 0.03, prefit= True)


In [79]:
select_X_train = selection.transform(X_train)

In [67]:
selection_model = xg_reg
selection_model.fit(select_X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [80]:
select_X_test = selection.transform(X_test)

In [81]:
y_pred = selection_model.predict(select_X_test)

In [82]:
rmse =mean_squared_error(y_test, y_pred, squared= False)
print('RMSE: ' f'{rmse}')

RMSE: 3.8842361441496993


In [83]:
data_dmatrix = xgb.DMatrix(data=select_X_train,label=y_train)

In [84]:
params = {"objective":"reg:linear",'subsample':0.7,'colsample_bytree': 0.7,'learning_rate': 0.3,'min_child_weight':6,
                'max_depth': 6, 'alpha': 10, 'silent':1,}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=6,
                    num_boost_round=5000,early_stopping_rounds=500,metrics="mae", as_pandas=True, seed=0)

[15:59:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:59:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost cor

In [85]:
cv_results.head()

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,2.838471,0.008069,2.841362,0.041945
1,2.697035,0.010247,2.702508,0.039687
2,2.659617,0.010871,2.669213,0.038640
3,2.653353,0.014920,2.666896,0.033870


In [86]:
print((cv_results["test-mae-mean"]).tail(1))

3    2.666896
Name: test-mae-mean, dtype: float64


In [87]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.09968858643213163